# Objetivo
    Cliente solicitou uma coleta de dados de todos os ginecologistas de MG dentro do site da Doctoralia. Foi solicitado Cidade, Nome e Telefone.
    link: https://www.doctoralia.com.br/pesquisa?q=Ginecologista&loc=Belo+Horizonte&filters%5Bspecializations%5D%5B0%5D=36

# Anotações

Manualmente foi feito um teste para ver quantas páginas tinham, como resultado, teve 82 páginas onde haviam perfis de doutores.

Notei que é melhor utilizar o BeautifulSoup do que o Selenium. Não encontrei no site um padrão fácil de coleta, logo, o WebScrapping foi feito com BeautifulSoup.

Não encontrei um padrão para navegar entre os perfis, logo, eu criei uma função(coletar) onde extrai todos os links do site, extraindo todos os perfis que eu preciso navegar.

Como é extraido todos os links do site, encontrei um padrão nos links de perfis e criei uma função(filtrar) para filtrar os links encontrados.

Coloquei uma coluna(link) para caso o cliente queira verificar manualmente o perfil.

# Algoritmo

## Módulos

In [ ]:
# Módulo de Requisições e Coleta.
from bs4 import BeautifulSoup
import requests

# Manipulação de Dados.
import pandas as pd

# Expressão Regular usada no código.
import re

# Exclusão de arquivos auxiliares.
import os

## Classe

In [ ]:
class Coleta():

    # Método contrutor.
    def __init__(self):

        # Formatação de dataframe para inserção dos dados.
        self.dataframe = pd.DataFrame(columns = ['Cidade', 'Nome', 'Telefone', 'Link'])

    # Função para extração de todos os links do site.
    def coletar(self, pagina):

        # A variável url foi desenvolvida para navegar nas páginas onde são encontrados os perfis.
        url = 'https://www.doctoralia.com.br/pesquisa?q=Ginecologista&loc=Belo+Horizonte&filters%5Bspecializations%5D%5B0%5D=36&page=' + str(pagina)
        response = requests.get(url)
        content = response.content
        soup = BeautifulSoup(content, 'html.parser')
        links = soup.find_all('a')

        # Criação de arquivo auxiliar. Arquivo da extração dos links.
        arquivo = open('links.txt', 'w')

        for link in links:
            href = link.get('href')
            arquivo.write(str(href) + '\n')

        arquivo.close()

    # Função para filtragem do conteúdo do arquivo auxiliar(links.txt), será extraído apenas links dos perfis.
    def filtrar(self):

        arquivo = open('links.txt', 'r')
        lista = []

        # Laço para filtragem dos links.
        for linha in arquivo:
            if 'ginecologista' in linha or 'clinicas/' in linha:
                if 'reviews' not in linha:
                    if linha not in lista:
                        conteudo = linha
                        if conteudo not in lista:
                            lista.append(linha)

        # Criação de arquivo auxiliar com filtragem dos links.
        arquivo2 = open('links-processados.txt', 'w')

        for x in lista:
            linha = x
            arquivo2.write(str(linha))

        arquivo2.close()
        arquivo.close()

    # Função para estruturar o .txt para lista em python para facilitar na utilização dos links.
    def listar_links(self):

        arquivo = open('links-processados.txt', 'r')
        lista = []

        for linha in arquivo:
            lista.append(linha)

        return lista

    # Função para coletar telefone, caso não seja encontrado ou não tenha, devolverá ponto de interrogação("?").
    def coletar_telefone(self, indice):

        try:
            lista = self.listar_links()
            url = lista[indice].strip()
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')
            telefone_element = soup.find('a', {'data-patient-app-event-name': 'dp-call-phone'})
            telefone = telefone_element.get_text(strip=True)

            return telefone
        
        except:

            return "?"
        
    # Função para coletar o nome. Caso seja perfil de doutor(ginecologista), a coleta é de uma forma, caso sejaa perfil de clínica, a coleta é diferente.
    def coletar_nome(self, link):

        # Se for doutor.
        if "ginecologista" in link:

            response = requests.get(link.strip())
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')
            elemento_span = soup.find('span', itemprop='name')
            nome = elemento_span.get_text(strip=True)
            return nome
        
        # Se for clínica.
        else:
            response = requests.get(link.strip())
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')
            elemento_h1 = soup.find('h1', class_='h3')
            nome = elemento_h1.get_text(strip=True)
            return nome

    # Função para coletar cidade. Caso não seja encontrado ou não tenha, devolverá ponto de interrogação("?").
    def coletar_cidade(self, link):

        try:

            if 'ginecologista' in link:

                response = requests.get(link.strip())
                html = response.content
                soup = BeautifulSoup(html, 'html.parser')
                elementos = soup.find_all('span', {'class': ['city', 'province region']})
                valores = [re.search(r'content="(.*?)"', str(elemento)).group(1) for elemento in elementos]
                juntas = []

                for i in range(0, len(valores), 2):
                    juntas.append(valores[i] + ', ' + valores[i+1])

                if len(juntas) == 1:
                    return juntas[0]
                
                else:
                    endereco_completo = ' / '.join(juntas)
                    enderecos_concatenados = [endereco_completo]

                    return enderecos_concatenados[0]
            else:

                response = requests.get(link)
                html = response.content
                soup = BeautifulSoup(html, 'html.parser')
                elemento_a = soup.find('a', class_='text-body font-weight-bold')
                endereco = elemento_a.get_text(strip=True)
                cidade = endereco.split(',')[0].strip()

                return cidade

        except:

            return '?'
    
    # Função de armazenagem dos dados coletados.
    def guardar_dados(self, cidade, nome, telefone, link):
        
        nova_linha = {'Cidade': cidade, 'Nome': nome, 'Telefone' : telefone, 'Link': link}
        self.dataframe = self.dataframe.append(nova_linha, ignore_index=True)

    # Função para status de coleta. Criei para saber o quanto o algoritmo coletou e o quantas páginas faltam.
    def contagem(self, pagina, itens_coletados):

        print(f'Página Atual: {pagina} --- Total de Coletas:{itens_coletados}')


    # Função principal de funcionamento da coleta, armazenagem e exportação da coleta de dados.
    def iniciar(self): 

        count = 0
        coletados = 0

        # Laço para navegar pelas 82 páginas.
        while count < 82:

            # Esse pedaço do código é para coletar, filtrar e estruturar links dos perfis.
            self.coletar(count)
            self.filtrar()
            lista = self.listar_links()

            # Laço para entrar em cada link e coletar os dados solicitados.
            for linha in range(len(lista)):    

                telefone = self.coletar_telefone(linha)
                nome = self.coletar_nome(lista[linha])
                link = lista[linha]
                endereco = self.coletar_cidade(lista[linha])

                self.guardar_dados(endereco, nome, telefone, link)
                coletados += 1
                self.contagem(count, coletados)

            count += 1
        
        # Exclusão dos arquivos auxiliares.
        os.remove('links.txt')
        os.remove('links-processados.txt')
        
        # Exportação dos dados
        self.dataframe.to_excel('dados.xlsx', index=True)

# Iniciar coleta
_ = Coleta()
_.iniciar()